In [ ]:
from __init__ import *

In [ ]:
ws, hs = read_ws_hs(
    "{}/mf".format(SETTING["output_directory_path"]),
    SETTING["model_mode"],
    SETTING["model_data_dicts"],
)

In [ ]:
distance_function = kraft.compute_information_distance_between_2_vectors

heat_map_layout_size = 640

layout = {"height": heat_map_layout_size, "width": heat_map_layout_size}

In [ ]:
w_node_x_node_distances = np.asarray(
    [
        kraft.apply_function_on_slices_from_2_dataframes(
            w,
            w,
            0,
            distance_function,
            layout={"title": {"text": "W{} Distance".format(i)}, **layout},
        ).values
        for i, w in enumerate(ws)
    ]
)

In [ ]:
h_node_x_node_distances = np.asarray(
    [
        kraft.apply_function_on_slices_from_2_dataframes(
            h,
            h,
            1,
            distance_function,
            layout={"title": {"text": "H{} Distance".format(i)}, **layout},
        ).values
        for i, h in enumerate(hs)
    ]
)

In [ ]:
node_x_node_distances = np.concatenate(
    (w_node_x_node_distances, h_node_x_node_distances)
)

node_x_node_distance = pd.DataFrame(
    np.mean(node_x_node_distances, axis=0), index=hs[0].index, columns=ws[0].columns
)

kraft.plot_heat_map(
    node_x_node_distance.iloc[
        kraft.cluster_matrix(node_x_node_distance.values, 0),
        kraft.cluster_matrix(node_x_node_distance.values, 1),
    ],
    layout={"title": {"text": "W H Distance"}, **layout},
)

In [ ]:
rs_01 = kraft.normalize_array(
    np.concatenate(tuple((r,) * r for r in SETTING["mf_rs"])), "0-1"
)

node_factors = SETTING["mf_r_range"] ** rs_01

node_marker_size = 3 ** (1 - rs_01) * 16

In [ ]:
gps_map_directory_path = "{}/gps_map".format(SETTING["output_directory_path"])

kraft.establish_path(gps_map_directory_path, "directory")

In [ ]:
for w_h_str in ("w", "h"):

    if w_h_str == "w":

        element_x_node = pd.concat(ws)

    else:

        element_x_node = pd.concat(hs, axis=1).T

    element_x_node *= node_factors

    element_x_node **= SETTING["{}_gps_map_node_power".format(w_h_str)]

    gps_map = kraft.GPSMap(
        element_x_node, node_x_node_distance, node_marker_size=node_marker_size
    )

    kraft.write_gps_map(
        "{}/{}_gps_map.pickle.gz".format(gps_map_directory_path, w_h_str), gps_map
    )

    gps_map.plot(
        element_trace=SETTING["{}_gps_map_element_trace".format(w_h_str)],
        html_file_path="{}/{}_gps_map.html".format(gps_map_directory_path, w_h_str),
    )